# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [37]:
import numpy as np
import pandas as pd
import requests
from scipy.stats import percentileofscore as score
import xlsxwriter
import math

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [38]:
stocks = pd.read_csv('sp_500_stocks.csv')
#stocks
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [39]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2389996386383,
 'week52high': 142.69,
 'week52low': 57.31,
 'week52change': 0.8624943851365688,
 'sharesOutstanding': 17554312516,
 'float': 0,
 'avg10Volume': 115841049,
 'avg30Volume': 109030021,
 'day200MovingAvg': 113.95,
 'day50MovingAvg': 128.37,
 'employees': 0,
 'ttmEPS': 3.41,
 'ttmDividendRate': 0.8164736040748191,
 'dividendYield': 0.006224576597211266,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-04',
 'nextEarningsDate': '0',
 'peRatio': 41.08894030018791,
 'beta': 1.171032789559101,
 'maxChangePercent': 51.31762312554708,
 'year5ChangePercent': 4.416398943337379,
 'year2ChangePercent': 2.5449944205722,
 'year1ChangePercent': 0.88536173039573,
 'ytdChangePercent': 0.8862955840538035,
 'month6ChangePercent': 0.5201150422903055,
 'month3ChangePercent': 0.18542864379482965,
 'month1ChangePercent': 0.157153242030333,
 'day30ChangePercent': 0.16149867711358,
 'day5ChangePercent': 0.02361604033686707}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [40]:
data['year1ChangePercent']

0.88536173039573

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [41]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [42]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ], 
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,123.76,0.401023,N/A
1,AAL,16.12,-0.454192,N/A
2,AAP,163.45,-0.002838,N/A
3,AAPL,140.33,0.885272,N/A
4,ABBV,106.02,0.248204,N/A
...,...,...,...,...
500,YUM,113.64,0.097814,N/A
501,ZBH,154.76,0.024649,N/A
502,ZBRA,396.44,0.487051,N/A
503,ZION,45.07,-0.143191,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [43]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
#Get stocks with 1-year price return in the top 50
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,37.850,2.180300,N/A
1,345,NVDA,537.330,1.214544,N/A
2,387,PYPL,236.290,1.151016,N/A
3,275,LB,38.500,1.138617,N/A
4,23,ALB,155.451,1.097694,N/A
5,31,AMD,93.820,0.981139,N/A
6,85,CDNS,138.060,0.950511,N/A
7,179,FCX,27.150,0.945032,N/A
8,339,NOW,573.500,0.916079,N/A
9,6,ABMD,327.610,0.911703,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [44]:
def input_portfolio_size():
    print('Enter the value of your portfolio, or enter \'exit\' to quit\t')
    portfolio_size = input()
    if(portfolio_size == 'exit'):
        print('Goodbye')
        return
    try:
        return float(portfolio_size)
        #print(float(portfolio_size))
    except ValueError:
        print('Please enter an integer')
        input_portfolio_size()

In [21]:
portfolio_size = input_portfolio_size()
position_size = portfolio_size/float(len(final_dataframe.index))

#No of shares to buy
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
print(f'Quantitative Momentum Strategy - Allocation for Equal weight S & P 500 fund with portfolio of size = \'{portfolio_size}\'')
final_dataframe

Enter the value of your portfolio, or enter 'exit' to quit	
1000000
Quantitative Momentum Strategy - Allocation for Equal weight S & P 500 fund with portfolio of size = '1000000.0'


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.728,2.184100,516
1,345,NVDA,537.020,1.229680,37
2,387,PYPL,237.370,1.126345,84
3,275,LB,38.585,1.115142,518
4,23,ALB,154.440,1.106312,129
5,31,AMD,93.880,0.970421,213
6,6,ABMD,328.180,0.933271,60
7,179,FCX,26.805,0.929217,746
8,85,CDNS,137.110,0.928719,145
9,339,NOW,574.770,0.920570,34


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [45]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
hqm_dataframe

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A'
                ], 
                index = hqm_columns
            ),
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,121.51,N/A,0.395479,N/A,0.358382,N/A,0.177153,N/A,0.027620,N/A
1,AAL,16.62,N/A,-0.456940,N/A,0.190996,N/A,0.307017,N/A,0.059176,N/A
2,AAP,158.68,N/A,-0.002800,N/A,0.101675,N/A,0.034570,N/A,0.046891,N/A
3,AAPL,138.90,N/A,0.885047,N/A,0.516453,N/A,0.186104,N/A,0.158277,N/A
4,ABBV,109.20,N/A,0.246058,N/A,0.119058,N/A,0.228239,N/A,-0.001817,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.59,N/A,0.095641,N/A,0.274460,N/A,0.215596,N/A,0.022038,N/A
501,ZBH,158.60,N/A,0.024402,N/A,0.322127,N/A,0.148950,N/A,0.018299,N/A
502,ZBRA,398.87,N/A,0.499663,N/A,0.514227,N/A,0.471916,N/A,0.002820,N/A
503,ZION,44.04,N/A,-0.140367,N/A,0.330990,N/A,0.509944,N/A,0.084250,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [46]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
#hqm_dataframe.index

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [48]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        percentile_col = f'{time_period} Return Percentile'
        change_col = f'{time_period} Price Return'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,121.51,N/A,0.395479,86.336634,0.358382,70.099010,0.177153,53.069307,0.027620,64.158416
1,AAL,16.62,N/A,-0.456940,2.574257,0.190996,39.801980,0.307017,79.603960,0.059176,80.198020
2,AAP,158.68,N/A,-0.002800,37.821782,0.101675,24.158416,0.034570,21.188119,0.046891,75.049505
3,AAPL,138.90,N/A,0.885047,98.019802,0.516453,88.514851,0.186104,55.445545,0.158277,97.227723
4,ABBV,109.20,N/A,0.246058,73.861386,0.119058,27.326733,0.228239,64.950495,-0.001817,40.000000
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.59,N/A,0.095641,52.673267,0.274460,55.445545,0.215596,62.376238,0.022038,60.792079
501,ZBH,158.60,N/A,0.024402,42.772277,0.322127,64.158416,0.148950,46.930693,0.018299,59.207921
502,ZBRA,398.87,N/A,0.499663,92.079208,0.514227,88.118812,0.471916,93.267327,0.002820,46.534653
503,ZION,44.04,N/A,-0.140367,19.603960,0.330990,65.742574,0.509944,95.049505,0.084250,88.316832


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: